In [1]:
from huggingface_hub import snapshot_download, hf_hub_download
import os

/Users/raghav.dave/Documents/raghav-dev-projects/audiotranscriber/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
repo_id_custom = "pyannote/wespeaker-voxceleb-resnet34-LM"
local_download_path = snapshot_download(repo_id=repo_id_custom, local_dir='models')
print(f"Custom model repo downloaded to: {local_download_path}")

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 15.43it/s]

Custom model repo downloaded to: /Users/raghav.dave/Documents/raghav-dev-projects/audiotranscriber/models


In [8]:
from pathlib import Path
from pyannote.audio import Pipeline
import whisper
from utils import words_per_segment

def load_pipeline_from_pretrained(path_to_config: str | Path) -> Pipeline:
    path_to_config = Path(path_to_config)

    print(f"Loading pyannote pipeline from {path_to_config}...")
    # the paths in the config are relative to the current working directory
    # so we need to change the working directory to the model path
    # and then change it back

    cwd = Path.cwd().resolve()  # store current working directory

    # first .parent is the folder of the config, second .parent is the folder containing the 'models' folder
    cd_to = path_to_config.parent.parent.resolve()

    print(f"Changing working directory to {cd_to}")
    os.chdir(cd_to)

    pipeline = Pipeline.from_pretrained(path_to_config)

    print(f"Changing working directory back to {cwd}")
    os.chdir(cwd)

    return pipeline

In [ ]:
PATH_TO_CONFIG = "models/pyannote_diarization_config.yaml"
pipeline = load_pipeline_from_pretrained(PATH_TO_CONFIG)

model = whisper.load_model("base")
diarization_result = pipeline("audio_chunks/Little ships recreate historic Dunkirk evacuation.wav")
transcription_result = model.transcribe("audio_chunks/Little ships recreate historic Dunkirk evacuation.wav", word_timestamps=True)

final_result = words_per_segment(transcription_result, diarization_result)

for _, segment in final_result.items():
    print(f'{segment["start"]:.3f}\t{segment["end"]:.3f}\t {segment["speaker"]}\t{segment["text"]}')


Loading pyannote pipeline from models/pyannote_diarization_config.yaml...
Changing working directory to /Users/raghav.dave/Documents/raghav-dev-projects/audiotranscriber
Changing working directory back to /Users/raghav.dave/Documents/raghav-dev-projects/audiotranscriber


/Users/raghav.dave/Documents/raghav-dev-projects/audiotranscriber/.venv/lib/python3.12/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)
/Users/raghav.dave/Documents/raghav-dev-projects/audiotranscriber/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyError: 'words'